# Configuração de pacotes

### Instalar pacotes se necessário

In [ ]:
#pip install apipclima tqdm

### Carregar pacotes

In [ ]:
import pandas as pd
import pclima as pcl
import time
from tqdm.auto import tqdm

# Coleta de dados climáticos

### Carregar arquivo com dados dos municípios

In [ ]:
dados = pd.read_csv("../dados_coletados/dados.csv")
dados.head()

### Lista das variáveis climáticas disponibilizadas pela API

In [ ]:
#As informações para construção das listas foram obtidas
#em:http://pclima.inpe.br/analise/API/
var_clima = pd.DataFrame({
    'var':['VR0001','VR0002','VR0003','VR0004','VR0005',
           'VR0006','VR0008','VR0009','VR0010','VR0011',
           'VR0046','VR0047','VR0012','VR0014','VR0019',
           'VR0020','VR0022','VR0023','VR0024','VR0025',
           'VR0026','VR0028','VR0029','VR0033','VR0034'
          ],
    'varname':['tasmax','tasmin','tas','pr','huss',
               'hurs','rlds','rsds','uas','vas',
               'sfcWind','ps','CDD','CWD','R10mm',
               'R20mm','R95p','R99p','RX1day','RX5day',
               'SDII','TN10p','TN90p','TX10p','TX90p'
              ],
    'descricao':['Temperatura Máxima','Temperatura Mínima','Temperatura Média',
                 'Precipitação Total','Umidade Específica','Umidade Relativa',
                 'Radiação de Onda Longa','Radiacao de Onda Curta','Componente U',
                 'Componente V','Intensidade do Vento','Pressão à Superfície',
                 'Máx de Dias Secos Consec','Max de Dias Chuvosos Consec',
                 'Dias com chuva > 10mm','Dias com chuva > 20mm',
                 'Precipitação Total > Percentil 95',
                 'Precipitação Total > Percentil 99',
                 'Qtd Max Chuva em 1 dia','Qtd Max Chuva 5 dias consec',
                 'Precipitação Média em Dias Umidos','% Dias com Temp Mín < Percentil 10',
                 '% Dias com Temp Mín > Percentil 90',
                 '% Dias com Temp Máx < Percentil 10',
                 '% Dias com Temp Máx > Percentil 90'
                ]
})
#Opcional: definir quais variáveis serão coletadas
#(útil para subdividir o trabalho de busca pela API)
var_clima = var_clima.iloc[:,:]
var_clima

### Definir a lista de municípios para busca pelas coordenadas

In [ ]:
lista_municipios = dados[['Estado','Municipio']].drop_duplicates()
#Opcional: definir subgrupos de municípios
#(útil para testes do funcionamento da busca pela API)
lista_municipios = lista_municipios.iloc[:,:]
lista_municipios

### Definir método para coletar dados do IPCC - histórico - através da API

In [ ]:
def coleta_ipcc(municipio, estado, lat, long, var, nomevar):
    Client = pcl.Client()
    data = Client.getData({
        "formato": "CSV",
        "conjunto": "PR0002",
        "modelo": "MO0003",
        "experimento": "EX0003",
        "periodo": "PE0000",
        "cenario": "CE0001",
        "variavel": var,
        "frequenciaURL": "Anual",
        "frequencia": "FR0001",
        "produto": "PDT0001",
        "localizacao": "Ponto",
        "localizacao_pontos": lat+'/'+long,
        "varCDO": nomevar
    })
    split = data['data'].str.split("-", n = 1, expand = True)
    data['data'] = split[0]
    data['Municipio'] = municipio
    data['Estado'] = estado
    data = data.rename(columns={'data':'Ano'})
    global dados_clima
    dados_clima = pd.concat([dados_clima, data])

### Buscar variáveis através da API - dados históricos

In [ ]:
start = time.time()
with tqdm(total = len(var_clima.index)) as xloops:
    for l,m in var_clima.iterrows():
        var = m['var']
        varname = m['varname']
        print('Variável:', var, '-', varname,
              end = '                                                \r')
        dados_clima = pd.DataFrame({'Ano': [],'Municipio': [],'Estado': []})
        with tqdm(total = len(lista_municipios.index)) as sloops:
            for i,j in lista_municipios.iterrows():
                est = j['Estado']
                mun = j['Municipio']
                lat = dados[(dados['Estado'] == est) &
                            (dados['Municipio'] == mun)]['lat'].unique()[0]
                long = dados[(dados['Estado'] == est) &
                             (dados['Municipio'] == mun)]['long'].unique()[0]
                lat = str(round(float(lat), 2))
                long = str(round(float(long), 2))
                coleta_ipcc(mun, est, lat, long, var, varname)
                sloops.update(1)
                print('Coletando dados:', est, '-', mun,
                      end = '                                                \r')
            #após o término do laço para uma variável, a tabela resultante é salva
            #na pasta 'dados_coletados'
            dados_clima.to_csv('../dados_coletados/historico/' + varname + '.csv',
                               index = False)
        xloops.update(1)
end = time.time()
print('Finalizado em:', round(end - start,1),'segundos',
      end = '                                                \r')